In [ ]:
# !apt-get update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
# !tar xf spark-2.3.1-bin-hadoop2.7.tgz
# !pip install -q findspark

# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

# !ls

# import findspark
# findspark.init()



In [ ]:
! pip install --upgrade pyspark
! pip install scikit-learn --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyspark
import pyspark.sql.functions as sqlfunc

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

# PySpark Core Concepts

In [ ]:
! wget https://mldataset0123.s3.amazonaws.com/original.csv

--2023-03-17 20:44:05--  https://mldataset0123.s3.amazonaws.com/original.csv
Resolving mldataset0123.s3.amazonaws.com (mldataset0123.s3.amazonaws.com)... 3.5.1.17, 52.216.115.43, 52.216.230.43, ...
Connecting to mldataset0123.s3.amazonaws.com (mldataset0123.s3.amazonaws.com)|3.5.1.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86874 (85K) [text/csv]
Saving to: ‘original.csv.1’

original.csv.1      100%[===================>]  84.84K   493KB/s    in 0.2s    

2023-03-17 20:44:05 (493 KB/s) - ‘original.csv.1’ saved [86874/86874]



## DataFrames


In [ ]:
# Reading the CSV file 

mydata = spark.read.format('csv').option('header','true').load('original.csv')

df = spark.read.csv('original.csv',header=True)

In [ ]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
# Defining a schema
from pyspark.sql.types import *

myschema = StructType([
    StructField('id',IntegerType()),
    StructField('first_name',StringType()),
    StructField('last_named',StringType()),
    StructField('gender',StringType()),
    StructField('City',StringType()),
    StructField('JobTitle',StringType()),
    StructField('Salary',StringType()),
    StructField('Latitude',StringType()),
    StructField('Longitude',IntegerType())
])


df3 = spark.read.csv('original.csv',header=True,schema=myschema)

df3.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+---------+
| id|first_name|last_named|gender|           City|            JobTitle|   Salary|  Latitude|Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+---------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075|     null|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|     null|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|     null|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212|     null|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null|     null|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145|     null|
|  7|     Masha|    Divers|Female|         Dachun|                null|$2

In [ ]:
df.dtypes

[('id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string')]

In [ ]:
df.head(10)

[Row(id='1', first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude='50.5774075', Longitude='16.4967184'),
 Row(id='2', first_name='Kimberly', last_name='Von Welden', gender='Female', City='Bulgan', JobTitle='Programmer II', Salary='$62846.60', Latitude='48.8231572', Longitude='103.5218199'),
 Row(id='3', first_name='Alvera', last_name='Di Boldi', gender='Female', City=None, JobTitle=None, Salary='$57576.52', Latitude='39.9947462', Longitude='116.3397725'),
 Row(id='4', first_name='Shannon', last_name="O'Griffin", gender='Male', City='Divnomorskoye', JobTitle='Budget/Accounting Analyst II', Salary='$61489.23', Latitude='44.5047212', Longitude='38.1300171'),
 Row(id='5', first_name='Sherwood', last_name='Macieja', gender='Male', City='Mytishchi', JobTitle='VP Sales', Salary='$63863.09', Latitude=None, Longitude='37.6489954'),
 Row(id='6', first_name='Maris', last_name='Folk', gender='Female', City='Kin

In [ ]:
df.first()

Row(id='1', first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude='50.5774075', Longitude='16.4967184')

In [ ]:
df.describe().show()

+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+-----------------+------------------+
|summary|               id|first_name|last_name|gender|               City|           JobTitle|   Salary|         Latitude|         Longitude|
+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+-----------------+------------------+
|  count|             1000|      1000|     1000|  1000|                999|                998|     1000|              999|              1000|
|   mean|            500.5|      null|     null|  null|               null|               null|     null|25.43151724234234|43.337564614499996|
| stddev|288.8194360957494|      null|     null|  null|               null|               null|     null| 24.5790825486909| 69.42064539970089|
|    min|                1|   Abagail|    Abbay|Female|             Abéché|Account Coordinator|$10101.92|       -0.6256517|        -0.4889547|

In [ ]:
df.columns

['id',
 'first_name',
 'last_name',
 'gender',
 'City',
 'JobTitle',
 'Salary',
 'Latitude',
 'Longitude']

In [ ]:
df.count()

1000

In [ ]:
df.distinct().count()

1000

In [ ]:
df.show(2)

+---+----------+----------+------+---------+-------------------+---------+----------+-----------+
| id|first_name| last_name|gender|     City|           JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------+-------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|Nowa Ruda|Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|   Bulgan|      Programmer II|$62846.60|48.8231572|103.5218199|
+---+----------+----------+------+---------+-------------------+---------+----------+-----------+
only showing top 2 rows



## Handling Null values

In [ ]:
df_dropped = df.na.drop()

df_dropped.show(5)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36|45.1905186|  0.7423124|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
only showing top 5 rows



In [ ]:
#Dropping null values from a specific column

from pyspark.sql.functions import * 

df_handled = df.withColumn('Clean City',when(df.City.isNull(),'unknown').otherwise(df.City))

df_handled.show(5)

+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+-------------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary|  Latitude|  Longitude|   Clean City|
+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+-------------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|       Bulgan|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.9947462|116.3397725|      unknown|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|    Mytishchi|
+---+----------+----------+------+-------------+----------------

### Drop Duplicate Values

In [ ]:
df_no_duplicate = df.dropDuplicates()
df_no_duplicate.show(5)

+---+----------+---------+------+-------+-------------------+---------+----------+----------+
| id|first_name|last_name|gender|   City|           JobTitle|   Salary|  Latitude| Longitude|
+---+----------+---------+------+-------+-------------------+---------+----------+----------+
|167|    Buiron|Franzotto|  Male|Prusice|   Health Coach III|$55731.78|51.3704296|16.9606267|
|610|    Waylen|O' Clovan|  Male|  Khyzy|  Marketing Manager|$73169.48|40.9109489|49.0729264|
|239|  Vivyanne| Astridge|Female|Baoshui|       Food Chemist|$11084.34| 30.513598|110.361715|
|256|    Stearn|   Niesel|  Male|Lungmar|Software Consultant|$31406.56|  28.85816|  89.94036|
|575|    Richie| Hellicar|  Male|Xichuan|       Developer IV|$40279.59|  33.13782|111.490964|
+---+----------+---------+------+-------+-------------------+---------+----------+----------+
only showing top 5 rows



In [ ]:
df_no_duplicate.count()

1000

## Selecting and filtering

In [ ]:
df.select('first_name','last_name').show(5)

+----------+----------+
|first_name| last_name|
+----------+----------+
|   Melinde| Shilburne|
|  Kimberly|Von Welden|
|    Alvera|  Di Boldi|
|   Shannon| O'Griffin|
|  Sherwood|   Macieja|
+----------+----------+
only showing top 5 rows



In [ ]:
df.withColumnRenamed('first_name','fn').show(5)


+---+--------+----------+------+-------------+--------------------+---------+----------+-----------+
| id|      fn| last_name|gender|         City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+--------+----------+------+-------------+--------------------+---------+----------+-----------+
|  1| Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|  Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.9947462|116.3397725|
|  4| Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
+---+--------+----------+------+-------------+--------------------+---------+----------+-----------+
only showing top 5 rows



In [ ]:
df.filter(df.first_name=='Melinde').show()

+---+----------+---------+------+---------+-------------------+---------+----------+----------+
| id|first_name|last_name|gender|     City|           JobTitle|   Salary|  Latitude| Longitude|
+---+----------+---------+------+---------+-------------------+---------+----------+----------+
|  1|   Melinde|Shilburne|Female|Nowa Ruda|Assistant Professor|$57438.18|50.5774075|16.4967184|
+---+----------+---------+------+---------+-------------------+---------+----------+----------+



In [ ]:
df.filter(df.first_name.like('%linde')).show()

+---+----------+---------+------+---------+-------------------+---------+----------+----------+
| id|first_name|last_name|gender|     City|           JobTitle|   Salary|  Latitude| Longitude|
+---+----------+---------+------+---------+-------------------+---------+----------+----------+
|  1|   Melinde|Shilburne|Female|Nowa Ruda|Assistant Professor|$57438.18|50.5774075|16.4967184|
+---+----------+---------+------+---------+-------------------+---------+----------+----------+



In [ ]:
df.filter(df.first_name.endswith('din')).show()

+---+----------+-------------+------+-----------+--------+---------+-----------+----------+
| id|first_name|    last_name|gender|       City|JobTitle|   Salary|   Latitude| Longitude|
+---+----------+-------------+------+-----------+--------+---------+-----------+----------+
|901|     Aldin|Matuszkiewicz|  Male|East London|Operator|$41468.83|-32.9549324|27.9319131|
+---+----------+-------------+------+-----------+--------+---------+-----------+----------+



In [ ]:
df.filter(df.first_name.startswith('All')).show()

+---+----------+-----------+------+----------+--------------------+---------+----------+-----------+
| id|first_name|  last_name|gender|      City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+-----------+------+----------+--------------------+---------+----------+-----------+
|115|    Allard|     Cordel|  Male| Mieścisko|Compensation Analyst|$18907.81|45.8630194|   5.947966|
|117|      Alli|    Mc Pake|Female|    Gerong|     Staff Scientist|$58172.73|-6.2002533|106.8221628|
|299|     Alley|Brockington|  Male| Balahovit|Community Outreac...|$74464.26|40.2544003| 44.6027766|
|347|   Allayne|      Dooly|  Male|Kastornoye|  Help Desk Operator|$23794.18|51.8315426| 38.1394579|
|743|     Allix|  Kilgallen|Female|Dahuangwei| Staff Accountant II|$66687.59| 39.534154| 113.087107|
+---+----------+-----------+------+----------+--------------------+---------+----------+-----------+



In [ ]:
df.filter(df.id.between(1,5)).show()

+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+



In [ ]:
df.filter(df.first_name.isin('Aldin','Valma')).show()

+---+----------+-------------+------+-----------+----------------+---------+-----------+-----------+
| id|first_name|    last_name|gender|       City|        JobTitle|   Salary|   Latitude|  Longitude|
+---+----------+-------------+------+-----------+----------------+---------+-----------+-----------+
|569|     Valma|      Bratton|Female|  Kurayoshi|Web Developer II|$32665.89| 35.4499037|133.7613311|
|901|     Aldin|Matuszkiewicz|  Male|East London|        Operator|$41468.83|-32.9549324| 27.9319131|
+---+----------+-------------+------+-----------+----------------+---------+-----------+-----------+



In [ ]:
df.select(df.first_name,df.first_name.substr(1,5).alias('name')).show()

+----------+-----+
|first_name| name|
+----------+-----+
|   Melinde|Melin|
|  Kimberly|Kimbe|
|    Alvera|Alver|
|   Shannon|Shann|
|  Sherwood|Sherw|
|     Maris|Maris|
|     Masha|Masha|
|   Goddart|Godda|
|      Roth| Roth|
|      Bran| Bran|
|    Kylynn|Kylyn|
|       Rey|  Rey|
|      Kerr| Kerr|
|    Mickie|Micki|
|    Kaspar|Kaspa|
|    Norbie|Norbi|
|    Claude|Claud|
|     Thain|Thain|
|  Tiffanie|Tiffa|
|    Ettore|Ettor|
+----------+-----+
only showing top 20 rows



In [ ]:
mydata.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
from pyspark.sql.functions import * 

mydata2= mydata.withColumn('clean_city',when(mydata.City.isNull(),'unknown').otherwise(mydata.City))

In [ ]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|        unknown|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Femal

In [ ]:
mydata2 = mydata2.filter(mydata2.JobTitle.isNotNull())

In [ ]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|Kinsealy-Drinan|
|  8|   Goddart|     Flear|  Mal

In [ ]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|Kinsealy-Drinan|
|  8|   Goddart|     Flear|  Mal

In [ ]:
mydata2.select('Salary').show()

+---------+
|   Salary|
+---------+
|$57438.18|
|$62846.60|
|$61489.23|
|$63863.09|
|$30101.16|
|$46116.36|
|$73697.10|
|$68098.42|
|$13604.63|
|$73423.70|
|$33432.99|
|$50838.53|
|$40163.03|
|$32492.73|
|$51862.48|
|$42135.67|
|$91925.08|
|$73921.33|
|$62755.85|
|$13708.03|
+---------+
only showing top 20 rows



In [ ]:
mydata2 = mydata2.withColumn('clean_salary',mydata2.Salary.substr(2,100).cast('float'))

In [ ]:
mean = mydata2.groupBy().avg('clean_salary').take(1)[0][0]

In [ ]:
mean

55516.32088199837

In [ ]:
from pyspark.sql.functions import lit


mydata2 = mydata2.withColumn('new_salary',when(mydata2.clean_salary.isNull(),lit(mean)).otherwise(mydata2.clean_salary))




In [ ]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|  61489.23046875|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 3

In [ ]:
latitudes  = mydata2.select('latitude').filter(mydata2.Latitude.isNotNull())

In [ ]:
latitudes.show()

+----------+
|  latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|53.4266145|
|45.1905186|
| 32.027934|
|  4.272793|
|     -5.85|
| 39.172378|
|49.8151822|
|42.1014803|
|49.7923299|
|43.4945737|
|52.7441662|
| 38.696249|
|-7.7232567|
|40.7172049|
|  49.16291|
|40.7576842|
|48.4902808|
+----------+
only showing top 20 rows



In [ ]:
latitudes = latitudes.withColumn('latitude2',latitudes.latitude.cast('float')).select('latitude2')

In [ ]:
latitudes.show()

+----------+
| latitude2|
+----------+
| 50.577408|
|  48.82316|
| 44.504723|
| 53.426613|
| 45.190517|
| 32.027935|
|  4.272793|
|     -5.85|
|  39.17238|
|  49.81518|
|  42.10148|
|  49.79233|
| 43.494576|
| 52.744167|
| 38.696247|
|-7.7232566|
| 40.717205|
|  49.16291|
| 40.757683|
|  48.49028|
+----------+
only showing top 20 rows



In [ ]:
import numpy as np 

median = np.median(latitudes.collect())

In [ ]:
median

31.93397331237793

In [ ]:
mydata2 = mydata2.withColumn('lat',when(mydata2.Latitude.isNull(),lit(median)).otherwise(mydata2.Latitude))

In [ ]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+-----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|              lat|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+-----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|       50.5774075|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|       48.8231572|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|  61489.23046875|     

In [ ]:
import pyspark.sql.functions as sqlfunc

genders = mydata2.groupBy('gender').agg(sqlfunc.avg('new_salary').alias('avgSalary'))

In [ ]:
genders.show()

+------+------------------+
|gender|         avgSalary|
+------+------------------+
|Female|55677.250125558036|
|  Male| 55361.09385573019|
+------+------------------+



In [ ]:
mydata2.filter(mydata.gender=='Male').select('gender','Salary').show()

+------+---------+
|gender|   Salary|
+------+---------+
|  Male|$61489.23|
|  Male|$63863.09|
|  Male|$46116.36|
|  Male|$73697.10|
|  Male|$68098.42|
|  Male|$33432.99|
|  Male|$50838.53|
|  Male|$40163.03|
|  Male|$32492.73|
|  Male|$42135.67|
|  Male|$73921.33|
|  Male|$62755.85|
|  Male|$13708.03|
|  Male|$55313.18|
|  Male|$78336.59|
|  Male|$72777.48|
|  Male|$53137.80|
|  Male|$34851.65|
|  Male|$99786.40|
|  Male|$36879.44|
+------+---------+
only showing top 20 rows



In [ ]:
df = mydata2.withColumn('female_salary',when(mydata2.gender =='Female',mydata2.new_salary).otherwise(lit(0)))

In [ ]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+-----------------+----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|              lat|   female_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+-----------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|       50.5774075|   57438.1796875|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|       48.8231572|   62846.6015625|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...

In [ ]:
df = mydata2.withColumn('female_salary',when(mydata2.gender =='Female',mydata2.new_salary).otherwise(lit(0)))
df = df.withColumn('male_salary',when(mydata2.gender =='Male',mydata2.new_salary).otherwise(lit(0)))

In [ ]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+-----------------+----------------+----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|              lat|   female_salary|     male_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+-----------------+----------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|       50.5774075|   57438.1796875|             0.0|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|       48.8231572|   62846.6015625|   

In [ ]:
df = df.groupby('jobTitle').agg(sqlfunc.avg('female_salary').alias("Female_average_salary"),sqlfunc.avg('male_salary').alias("Male_average_salary"))

In [ ]:
df.show()

+--------------------+---------------------+-------------------+
|            jobTitle|Female_average_salary|Male_average_salary|
+--------------------+---------------------+-------------------+
|Systems Administr...|      50590.474609375|   15540.9501953125|
|   Media Manager III|   29586.436197916668| 17381.920572916668|
|  Recruiting Manager|   34848.452473958336|   26383.4951171875|
|       Geologist III|         31749.046875|     12830.75390625|
|        Geologist II|                  0.0|    43293.865234375|
|Database Administ...|                  0.0|      52018.4609375|
|   Financial Analyst|      23353.776953125|        39606.05625|
|  Analyst Programmer|     16406.1287109375|   21042.9634765625|
|Software Engineer II|                  0.0|       74782.640625|
|       Accountant IV|      82732.248046875|                0.0|
|    Product Engineer|       41825.48359375|        20464.94375|
|Software Test Eng...|     32218.6083984375|    27122.462890625|
|Safety Technician...|   

In [ ]:
df = df.withColumn('delta',df.Female_average_salary-df.Male_average_salary)

df.show()

+--------------------+---------------------+-------------------+-------------------+
|            jobTitle|Female_average_salary|Male_average_salary|              delta|
+--------------------+---------------------+-------------------+-------------------+
|Systems Administr...|      50590.474609375|   15540.9501953125|   35049.5244140625|
|   Media Manager III|   29586.436197916668| 17381.920572916668|       12204.515625|
|  Recruiting Manager|   34848.452473958336|   26383.4951171875|  8464.957356770836|
|       Geologist III|         31749.046875|     12830.75390625|     18918.29296875|
|        Geologist II|                  0.0|    43293.865234375|   -43293.865234375|
|Database Administ...|                  0.0|      52018.4609375|     -52018.4609375|
|   Financial Analyst|      23353.776953125|        39606.05625|   -16252.279296875|
|  Analyst Programmer|     16406.1287109375|   21042.9634765625| -4636.834765625001|
|Software Engineer II|                  0.0|       74782.640625| 

In [ ]:
cityavg = mydata2.groupBy('city').agg(sqlfunc.avg('new_salary').alias("City_avg_salary"))

cityavg = cityavg.sort(col('City_avg_salary').desc())

In [ ]:
cityavg.show()

+-----------------+---------------+
|             city|City_avg_salary|
+-----------------+---------------+
|        Mesopotam|    99948.28125|
|       Zhongcheng|   99942.921875|
|           Caxias|  99786.3984375|
|      Karangtawar|  99638.9921875|
|        Itabaiana|    99502.15625|
|           Pasian|    99421.34375|
|           Webuye|   99368.546875|
|      Yuktae-dong|   99250.828125|
|           Zinder|    99222.84375|
|   Timiryazevskiy|     99142.9375|
|        Sawahbaru|  99013.7109375|
|          Madimba|  98737.8671875|
|         Huangshi|    98690.34375|
|          Gharyan|     98679.3125|
|         Yŏnan-ŭp|   98628.609375|
|     Wringinputih|  98603.8203125|
|Monte da Boavista|    98586.71875|
|          Klukeng|  98439.4921875|
|         Murmashi|    98226.15625|
|        Fox Creek|        98138.0|
+-----------------+---------------+
only showing top 20 rows



## Multiple Filters

In [ ]:
mydata.show(10)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
df_filter = mydata.filter((mydata.first_name.isin('Aldin','Valma'))&(mydata.City.like('%ondon')))

In [ ]:
df_filter.show()

+---+----------+-------------+------+-----------+--------+---------+-----------+----------+
| id|first_name|    last_name|gender|       City|JobTitle|   Salary|   Latitude| Longitude|
+---+----------+-------------+------+-----------+--------+---------+-----------+----------+
|901|     Aldin|Matuszkiewicz|  Male|East London|Operator|$41468.83|-32.9549324|27.9319131|
+---+----------+-------------+------+-----------+--------+---------+-----------+----------+



In [ ]:
df_filter = mydata.filter((mydata.id>10)&(mydata.id<100))

In [ ]:
df_filter.show()

+---+----------+---------+------+--------------+--------------------+---------+----------+-----------+
| id|first_name|last_name|gender|          City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+--------------+--------------------+---------+----------+-----------+
| 11|    Kylynn|  Lockart|Female|      El Cardo|Nuclear Power Eng...|$13604.63|     -5.85|-79.8833329|
| 12|       Rey|   Meharg|Female|   Wangqingtuo|Systems Administr...|$73423.70| 39.172378| 116.931607|
| 13|      Kerr|   Braden|  Male|     Sułkowice|Compensation Analyst|$33432.99|49.8151822| 19.3771749|
| 14|    Mickie|Whanstall|  Male|   Springfield|Assistant Media P...|$50838.53|42.1014803|-72.5766759|
| 15|    Kaspar|    Pally|  Male|        Chrást|  Analyst Programmer|$40163.03|49.7923299| 13.4915324|
| 16|    Norbie|   Gwyllt|  Male|        Xijiao|              Editor|$32492.73|43.4945737|  5.8978018|
| 17|    Claude|   Briant|Female|     Mieścisko|Research Assistan...|$518

## Value Counts

In [ ]:
mydata.groupBy('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|Female|  492|
|  Male|  508|
+------+-----+



## Read df from sql 

In [ ]:
mydata.registerTempTable("original")


/usr/local/lib/python3.9/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [ ]:
query1 = spark.sql("select * from original")

In [ ]:
query1.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
query2 = spark.sql('select concat(first_name," ",last_name) as full_name from original where gender = "Female"')

In [ ]:
query2.show()

+-------------------+
|          full_name|
+-------------------+
|  Melinde Shilburne|
|Kimberly Von Welden|
|    Alvera Di Boldi|
|         Maris Folk|
|       Masha Divers|
|     Kylynn Lockart|
|         Rey Meharg|
|      Claude Briant|
|  Tiffanie Pattison|
|    Lurleen Janczak|
|      Nichol Holtum|
|       Shaun Bridle|
|     Leandra Anfrey|
|    Jaquelyn Hazard|
|  Prudence Honacker|
|       Cherey Liger|
|          Neda Krop|
|    Barbi Fattorini|
|   Lonnie Townshend|
|    Valida Salzberg|
+-------------------+
only showing top 20 rows



## Calculated Columns

In [ ]:
mydata = mydata.withColumn('clean_salary',mydata.Salary.substr(2,100).cast('float'))

In [ ]:
mydata.show(2)

+---+----------+----------+------+---------+-------------------+---------+----------+-----------+------------+
| id|first_name| last_name|gender|     City|           JobTitle|   Salary|  Latitude|  Longitude|clean_salary|
+---+----------+----------+------+---------+-------------------+---------+----------+-----------+------------+
|  1|   Melinde| Shilburne|Female|Nowa Ruda|Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18|
|  2|  Kimberly|Von Welden|Female|   Bulgan|      Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|
+---+----------+----------+------+---------+-------------------+---------+----------+-----------+------------+
only showing top 2 rows



In [ ]:
mydata = mydata.withColumn('monthly_salary',mydata.clean_salary/12)

In [ ]:
mydata.show(10)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+------------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|clean_salary|    monthly_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+------------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18| 4786.514973958333|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|    5237.216796875|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|    57576.52| 4798.043294270833|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|    61489.23|   5124.1025390625|
|  5|  Sherwood|   Macieja|  Male|      M

In [ ]:
mydata = mydata.withColumn('are_they_female',when(mydata.gender=='Female','yes').otherwise('no'))

In [ ]:
mydata.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+------------------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|clean_salary|    monthly_salary|are_they_female|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+------------------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18| 4786.514973958333|            yes|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|    5237.216796875|            yes|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|    57576.52| 4798.043294270833|            yes|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|4

## Data Aggregation

In [ ]:
df1 = mydata.groupBy('gender').count()
df1.show()

+------+-----+
|gender|count|
+------+-----+
|Female|  492|
|  Male|  508|
+------+-----+



In [ ]:
df1 = mydata.groupBy('gender').agg(sqlfunc.sum('clean_salary'))
df1.show()

+------+--------------------+
|gender|   sum(clean_salary)|
+------+--------------------+
|Female|2.7364519950195312E7|
|  Male|2.8123435678710938E7|
+------+--------------------+



In [ ]:
df2 = mydata.groupBy('gender').agg(
    sqlfunc.sum('clean_salary').alias('total'),
    sqlfunc.avg('clean_salary').alias('Average'),
    sqlfunc.min('clean_salary').alias('min'),
    sqlfunc.max('clean_salary').alias('max'),
)

df2.show()

+------+--------------------+-----------------+--------+--------+
|gender|               total|          Average|     min|     max|
+------+--------------------+-----------------+--------+--------+
|Female|2.7364519950195312E7|55618.94298820185|10616.44|99948.28|
|  Male|2.8123435678710938E7|55361.09385573019|10101.92|99942.92|
+------+--------------------+-----------------+--------+--------+



In [ ]:
df2 = mydata.groupBy('gender','city').agg(
    sqlfunc.sum('clean_salary').alias('total'),
    sqlfunc.avg('clean_salary').alias('Average'),
    sqlfunc.min('clean_salary').alias('min'),
    sqlfunc.max('clean_salary').alias('max'),
)

df2.show()

+------+-----------------+----------------+----------------+--------+--------+
|gender|             city|           total|         Average|     min|     max|
+------+-----------------+----------------+----------------+--------+--------+
|Female|           Dachun| 25090.869140625| 25090.869140625|25090.87|25090.87|
|Female|      Trollhättan|106623.369140625|53311.6845703125|26830.47| 79792.9|
|  Male|          Wenshao| 18941.509765625| 18941.509765625|18941.51|18941.51|
|Female|            Lanas| 13765.900390625| 13765.900390625| 13765.9| 13765.9|
|  Male|            Mörön|    77940.078125|    77940.078125|77940.08|77940.08|
|Female|             Same|   73369.7265625|   73369.7265625|73369.73|73369.73|
|Female|          Sawahan|  24608.83984375|  24608.83984375|24608.84|24608.84|
|  Male|Monte da Boavista|     98586.71875|     98586.71875|98586.72|98586.72|
|Female|         Nusajaya|    71637.921875|    71637.921875|71637.92|71637.92|
|Female|            Kista|   96192.3984375|   96192.

## Writing DataFrame to Files

In [ ]:
df2.write.csv('df2.csv')
df2.write.json('df2.json')
df2.write.parquet('df2.parquet')

# Challenge

In [ ]:
! wget https://mldataset0123.s3.amazonaws.com/challenge.csv

--2023-03-17 21:53:01--  https://mldataset0123.s3.amazonaws.com/challenge.csv
Resolving mldataset0123.s3.amazonaws.com (mldataset0123.s3.amazonaws.com)... 54.231.170.209, 52.216.214.161, 3.5.11.184, ...
Connecting to mldataset0123.s3.amazonaws.com (mldataset0123.s3.amazonaws.com)|54.231.170.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38621 (38K) [text/csv]
Saving to: ‘challenge.csv’

challenge.csv       100%[===================>]  37.72K  --.-KB/s    in 0.09s   

2023-03-17 21:53:02 (440 KB/s) - ‘challenge.csv’ saved [38621/38621]



In [ ]:
df = spark.read.csv('challenge.csv',header=True)

In [ ]:
df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [ ]:
df = df.withColumn('mexico',when(df.Country=='Mexico','YES').otherwise('NO') )

In [ ]:
df.show()

+---------------+--------------+-----------------+----------+------+
|     ip_address|       Country|      Domain Name|Bytes_used|mexico|
+---------------+--------------+-----------------+----------+------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|    NO|
| 119.239.207.13|         China|         youtu.be|        51|    NO|
|  68.69.217.210|         China|        adobe.com|        10|    NO|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|    NO|
|   211.13.10.68|     Indonesia|          hud.gov|        29|    NO|
|   239.80.21.97|      Suriname|       smh.com.au|       218|    NO|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|    NO|
| 127.242.24.138|         China| surveymonkey.com|       123|    NO|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|    NO|
|   237.54.11.63|         China|       amazon.com|        83|    NO|
| 252.141.157.25|         Japan|      cornell.edu|       374|    NO|
|185.220.128.248|       Belgium|  

In [ ]:
df2 = df.groupBy('mexico').agg(
    sqlfunc.sum('Bytes_used').alias('Bytes_used')
)

df2.show()


+------+----------+
|mexico|Bytes_used|
+------+----------+
|   YES|    6293.0|
|    NO|  508076.0|
+------+----------+



In [ ]:
df3 = df.groupBy('Country').agg(
    sqlfunc.countDistinct('ip_address').alias('ip_address')
)

df3.sort(col('ip_address').desc()).show()


+--------------+----------+
|       Country|ip_address|
+--------------+----------+
|         China|       172|
|     Indonesia|       114|
|   Philippines|        65|
|        Russia|        56|
|        Brazil|        35|
|        Poland|        31|
|        Sweden|        28|
|         Japan|        25|
|Czech Republic|        23|
|      Portugal|        23|
|        France|        21|
|          Peru|        19|
|      Colombia|        17|
| United States|        15|
|     Argentina|        14|
|       Ukraine|        14|
|        Mexico|        13|
|      Thailand|        12|
|       Nigeria|        11|
|        Canada|        11|
+--------------+----------+
only showing top 20 rows

